In [1]:
import torchrl
import torchrl.network as network
from torchrl.a2c import A2C
from torchrl.a2c import ActorCritic

# later incorporate this into torchrl
from torchrl.utils import make_atari_env
from torchrl.utils import weights_init
from torchrl.parallel import SubprocVecEnv
from torchrl.parallel import VecFrameStack

import torch
import torch.optim as optim
import numpy as np
from tensorboardX import SummaryWriter


In [2]:
params = {}
params["num_in_channels"] = 4
params["num_latent_nodes"] = 512
params["num_workers"] = 8
params["use_cuda"] = True

In [3]:
env = make_atari_env("BreakoutNoFrameskip-v4", params["num_workers"])

Process Process-5:
Process Process-7:
Process Process-8:
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/will/anaconda3/lib/python3.7/multiprocessing/process.py", 

In [4]:
env = VecFrameStack(env, 4)

In [5]:
params["num_actions"] = env.action_space.n

# hyperparaters for traing
params["discount_gamma"] = 0.9
params["entropy_coef"] = 0.01
params["value_coef"] = 0.5
params["use_gae"] = True
params["gae_tau"] = 0.95

params["max_grad_norm"] = 0.5
params["learning_rate"] = 7e-4
params["RMS_alpha"] = 0.99
params["RMS_eps"] = 1e-5
params["rollout_len"] = 5
params["resume"] = False

In [6]:
# define a model and its optimizer
model = ActorCritic(params)
model.apply(weights_init)

# RMSprop better for RL
optimizer = optim.RMSprop(model.parameters(),
                           params["learning_rate"],
                           alpha=params["RMS_alpha"],
                           eps=params["RMS_eps"]
                          )

In [7]:
agent = A2C(env, model, optimizer, params)

In [8]:
if params["resume"]:
    checkpoint = torch.load("Breakout/model0.pth")
    model.load_state_dict(checkpoint["state_dict"])
    agent.num_steps = checkpoint["num_steps"]

In [9]:
writer = SummaryWriter()

In [10]:
agent.curr_states = env.reset()

In [18]:
batch_size = int(agent.num_workers * agent.rollout_len)
epoch = int(4e7 // batch_size)

In [20]:
model.cuda()
model.train()

for e in range(epoch):
    agent.train_step()
    
    if e % 10 == 1:
        writer.add_scalar('policy loss', agent.policy_loss, agent.num_steps)
        writer.add_scalar('entropy loss', agent.entropy_loss, agent.num_steps)
        writer.add_scalar('value loss', agent.value_loss, agent.num_steps)
        writer.add_scalar('Mean Reward',  np.mean(agent.online_rewards), agent.num_steps)
        
        if len(agent.episode_rewards) > 0:
            writer.add_scalar('Episode Reward', np.mean(agent.episode_rewards), agent.num_steps)
    
    # save the model every 1000000 steps
    if e % (1e6 // batch_size) == 0:
        checkpoint = {
            'num_steps': agent.num_steps,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict()
            }
        torch.save(checkpoint, "Breakout/saved_model" + str(agent.num_steps) + ".pth")
        

KeyboardInterrupt: 